# Atomata results analysis

## Read from database

In [1]:
%load_ext autoreload
%autoreload 2
    
import pandas as pd
import sqlite3

from python.repository import state_vectors_by_run_id, total_runs, parameters_by_run_id
from python.metric import empirical_probability, complexity, emergence, self_organization

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("results.db3")

## Run with max. emergence

In [ ]:
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor

# Step 1: Function to process each run
def process_run(run_id, con, runs_count):
    print(f"Computing results for run {run_id}")
    con = sqlite3.connect("results.db3")
    state_vectors = state_vectors_by_run_id(run_id, con)
    
    probability_mass = state_vectors["count"].map(lambda state_count: empirical_probability(state_count, runs_count))
    cx = complexity(probability_mass)
    return {"run_id": run_id, "complexity": cx}

# Step 2: Main function to parallelize
def parallelized_processing(con):
    total_runs_df = total_runs(con)
    runs_count = total_runs_df.iloc[0]["MAX(run_id)"]
    print(f"Total runs to compute: {runs_count}")

    results = []
    
    # Use ThreadPoolExecutor for parallel execution
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_run, run_id, con, runs_count) for run_id in range(1, runs_count + 1)]
        for future in concurrent.futures.as_completed(futures):
            # Gather results as each thread completes
            results.append(future.result())
    
    return results

# Step 3: Run the parallelized process
results = parallelized_processing(con)
    

In [2]:
max_run_id = 902
max_complexity = 0
for result in results:
    if result["complexity"] > max_complexity:
        max_complexity = result["complexity"]
        max_run_id = result["run_id"]

NameError: name 'results' is not defined

In [5]:
max_parameters = parameters_by_run_id(max_run_id, con)
max_parameters

,run_id,amount,border,timestep,gravity_constant,friction,max_velocity,bucket_size,created_at
0,902,100,400.0,0.0002,0.5,0.005,20000.0,5.0,2024-08-29 16:43:11
